In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

In [2]:
import numpy as np
myint = np.int8

In [3]:
# import numexpr as ne

In [4]:
from itertools import starmap, product, combinations, chain, permutations

In [5]:
from funcy import *

In [6]:
from functools import reduce

In [7]:
from tqdm import tqdm

from joblib import Parallel, delayed, Memory

J = -1
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def par(gen_expr, j=None, backend=None, verbose=None, prefer=None):
    if j is None:
        j = J
    if backend is None:
        backend = BACKEND
    if verbose is None:
        verbose = V
    if prefer is None:
        prefer = PREFER
    return Parallel(n_jobs=j, backend=backend, verbose=verbose, prefer=prefer)(gen_expr)

def identity(x):
    return x

In [8]:
from random import choice

In [9]:
CAREFUL = False

In [10]:
import sparse

In [11]:
from scipy.special import binom#, comb

In [12]:
import os

In [13]:
import torch

In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    
    print(torch.cuda.get_device_name(0))
    total_mem_MB = torch.cuda.get_device_properties(device).total_memory / 1e6
    print('Total Memory: {0}'.format(total_mem_MB) )
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2070
Total Memory: 8367.439872
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [14]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

gpu_int8_ttype = torch.cuda.CharTensor
gpu_int16_ttype = torch.cuda.ShortTensor

my_ttype = gpu_int8_ttype
my_dtype = torch.uint8

def t(ndarray):
    if ndarray.dtype == myint:
        return torch.tensor(ndarray.astype(np.int16)).type(my_ttype)
    return torch.tensor(ndarray).type(my_ttype)

# Read in (or make) object vectors

## Make

In [15]:
m = 5

In [16]:
max_num_objects = 2 ** m
max_num_objects

max_num_partial_fvs = (2 + 1) ** m
max_num_partial_fvs

32

243

In [17]:
def make_random_pfv():
    return np.random.randint(3, size=m, dtype=myint) - 1

In [18]:
max_num_objects
actual_num_objects = np.random.randint(max_num_objects)
# actual_num_objects = 40
actual_num_objects

assert actual_num_objects < max_num_objects

32

1

In [19]:
def zeroToMinusOne(u):
    return np.array([x if x == 1 else -1 for x in u])

def makeRandomObjects(l, num_features, as_ndarray=False):
    l = actual_num_objects
    m = num_features
    objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
    objects = tuple(map(np.array, objects))
    objects = tuple([zeroToMinusOne(o) for o in objects])
    if not as_ndarray:
        return objects
    return np.array(objects)

# objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
# objects = tuple(map(np.array, objects))
# objects = tuple([zeroToMinusOne(o) for o in objects])
objects = makeRandomObjects(actual_num_objects, m)
l = len(objects)



actual_num_objects = len(objects)
actual_num_objects
objects

1

(array([ 1,  1,  1, -1,  1]),)

In [20]:
objectMap = np.array(objects) #np.array([objects[i] for i in range(l)])
objectMap.shape
objectMap
objectMap[0]

O = objectMap

(1, 5)

array([[ 1,  1,  1, -1,  1]])

array([ 1,  1,  1, -1,  1])

## Read-in

In [21]:
%ls *.npy

brh.npy  hayes.npy


In [22]:
objectMap = np.load('brh.npy')
objectMap.shape

l, m = objectMap.shape
actual_num_objects = l

O = objectMap
objects = tuple(objectMap)

(91, 23)

In [23]:
max_num_objects = 2 ** m
max_num_objects
actual_num_objects / max_num_objects

max_num_partial_fvs = (2 + 1) ** m
# max_num_partial_fvs
'{:2,}'.format(max_num_partial_fvs)
'{:2E}'.format(max_num_partial_fvs)

8388608

1.0848045349121094e-05

'94,143,178,827'

'9.414318E+10'

# Operations 

## Make generator vectors

In [24]:
def make_generator_vectors(num_features):
    basis_vectors = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    basis_vectors_neg = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    for i,v in enumerate(basis_vectors):
        v[i] = 1
    for i,v in enumerate(basis_vectors_neg):
        v[i] = -1
    generators = basis_vectors + basis_vectors_neg
    return generators

In [25]:
generators = make_generator_vectors(m)
generators

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0,

In [26]:
# max_num_objects = 2 ** m
# max_num_objects

# max_num_partial_fvs = (2 + 1) ** m
# max_num_partial_fvs

## Boilerplate

In [27]:
def wf_pfv(v):
    '''
    Indicates whether v is a well-formed partially-specified feature vector.
    '''
    allowedValues = {-1,0,1}
    return all([x in allowedValues for x in v])

In [28]:
def wf_tfv(v):
    '''
    Indicates whether v is a well-formed totally-specified feature vector.
    '''
    allowedValues = {-1,1}
    return all([x in allowedValues for x in v])

In [29]:
def uniquify(ndarray_iterable):
    tuples = [tuple(a) for a in ndarray_iterable]
    s = set(tuples)
    arrays = [np.array(t) for t in s]
    return arrays

## Upper and lower closures of a partially-specified feature vector

In [30]:
upset_size_for_fsfvs = np.sum([binom(m, i) for i in np.arange(1,m)]); upset_size_for_fsfvs
"{:,}".format(upset_size_for_fsfvs)
"{:.2E}".format(upset_size_for_fsfvs)

8388606.0

'8,388,606.0'

'8.39E+06'

In [31]:
def put_(a, ind, v, mode='raise'):
    '''
    A functional version of np.put that returns the array it operates on. 
    See the documentation for that function for more details.
    '''
    np.put(a=a, ind=ind, v=v, mode=mode)
    return a


def upper_closure(x):
    '''
    The upper closure ↑x of a pfv x is the set of strictly less specified vectors.
    This function returns that as a generator.
    
    WARNING: There are O(𝚺_i=1^i=m m choose i) elements in this set.
    '''
    specified_indices = x.nonzero()[0]
    m_x = len(specified_indices)
    #There is one element in ↑x for each possible combination of specified indices.
    combinations_of_indices_to_unspecify = cat(combinations(specified_indices, i)
                                               for i in range(1,m_x))
    up_x = (put_(x.copy(), tuple(ind), 0) for ind in combinations_of_indices_to_unspecify)
    return up_x


def lower_closure(x):
    '''
    The lower closure ↓x of a pfv x is the set of strictly more specified vectors.
    This function returns that as a generator.
    
    WARNING: There are O(𝚺_i=1^i=m choose(m,i) * 2^i) elements in this set.
    '''
    unspecified_indices = (x == 0).nonzero()[0]
    m_x = len(unspecified_indices)
    #There are 2^i elements in ↓x for each possible combination of i unspecified indices.
    combinations_of_indices_to_specify = cat(combinations(unspecified_indices, i)
                                             for i in range(1,m_x))
#     specifications = cat(map(np.array, permutations([-1,1], len(combo)))
#                          for combo in combinations_of_indices_to_specify)
    down_x = (put_(x.copy(), tuple(ind), spec) 
              for ind in combinations_of_indices_to_specify
              for spec in map(np.array, 
                              product([-1,1], repeat=len(ind))))
    return down_x


def gen_uc(x):
    '''
    Generates a random element u of ↑x.
    Generative procedure:
      1. A number n of indices to unspecify is chosen uniformly from among specified ones.
      2. n indices are sampled without replacement from among the specified ones.
    '''
    specified_indices = x.nonzero()[0]
    m_x = len(specified_indices)
    num_indices_to_unspecify = choice(np.arange(1,m_x))
#     assert num_indices_to_unspecify > 0
    indices_to_unspecify = np.random.choice(specified_indices, 
                                            size=num_indices_to_unspecify, 
                                            replace=False)
    u = put_(x.copy(), indices_to_unspecify, 0)
    return u


def gen_lc(x):
    '''
    Generates a random element l of ↓x.
    Generative procedure:
      1. A number n of indices to specify is chosen uniformly from among unspecified ones.
      2. n indices are sampled without replacement from among the unspecified ones.
    '''
    unspecified_indices = (x == 0).nonzero()[0]
    m_x = len(unspecified_indices)
    num_indices_to_specify = choice(np.arange(1,m_x))
#     assert num_indices_to_specify > 0
    indices_to_specify = np.random.choice(unspecified_indices, 
                                         size=num_indices_to_specify, 
                                         replace=False)
    possible_specifications = lmap(np.array, product([-1,1], repeat=len(indices_to_specify)))
    if len(possible_specifications) == 0:
        print(x, m_x, unspecified_indices, num_indices_to_specify, indices_to_specify)
    spec = choice(possible_specifications)
    l = put_(x.copy(), indices_to_specify, spec)
    return l


def gen_agreeing(x):
    '''
    Generates a random psfv vector r that agrees with x.
    '''
    specified_indices = x.nonzero()[0]
    unspecified_indices = (x == 0).nonzero()[0]
    has_uc = len(specified_indices) > 0
    has_lc = len(unspecified_indices) > 0
    if has_uc and has_lc:
        sample_function = choice([gen_uc, gen_lc])
        return sample_function(x)
    elif has_uc:
        return gen_uc(x)
    elif has_lc:
        return gen_lc(x)
    else:
        raise Exception(f'x has neither an upper nor a lower closure:\n\tx = {x}')

In [32]:
r = choice(objects); r

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

In [33]:
r
ucr = upper_closure(r)
ucr_l = list(ucr)
len(ucr_l)
ucr_l[0]
choice(ucr_l)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

524286

array([ 0, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

array([ 1, -1,  0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0, -1, -1,  0,  0,  0], dtype=int8)

In [34]:
r
lcr = lower_closure(r)
lcr_l = list(lcr)
len(lcr_l)
lcr_l[0]
choice(lcr_l)
choice(lcr_l)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

64

array([ 1, -1,  1,  1, -1,  1,  1,  1, -1,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  1,  1, -1,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  1,  1,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

In [35]:
r
gen_uc(r)
gen_lc(r)
gen_agreeing(r)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

array([ 1,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  1, -1, -1,  1,  0,
        0,  0,  0,  0,  0,  1], dtype=int8)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  1,  0,  0,  1,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

array([ 1, -1,  1,  1, -1,  1,  1,  1,  1,  0,  0,  0,  1, -1, -1,  1,  1,
       -1, -1, -1, -1,  1,  1], dtype=int8)

## Agreement

In [36]:
def ag(x,y):
    '''
    Formula:
    (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y)), where T = 1 and F = 0
    
    Pattern:
    x = x ⟶ 1
    0 = _ ⟶ 1
    _ = 0 ⟶ 1
    _ = _ ⟶ 0
    '''
    if x == y:
        return True
    elif x == 0:
        return True
    elif y == 0:
        return True
    else:
        return False

In [150]:
def ag_(x,y):
    return not x*y == -1

In [152]:
ag(0,0) == ag_(0,0)
ag(0,1) == ag_(0,1)
ag(0,-1) == ag_(0,-1)
ag(-1,0) == ag_(-1,0)
ag(-1,1) == ag_(-1,1)
ag(-1,-1) == ag_(-1,-1)
ag(1,0) == ag_(1,0)
ag(1,1) == ag_(1,1)
ag(1,-1) == ag_(1,-1)

True

True

True

True

True

True

True

True

True

In [37]:
def agree(u,v):
    '''
    Given two vectors u and v, returns a binary vector indicating,
    elementwise, whether u and v 'agree'.
    
    agree(u[i], v[i]) iff (u[i] == 0 or v[i] == 0) or (u[i] == v[i])
    '''
#     return np.array([True if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else False 
#                      for i in range(len(u))])
    return np.array([1 if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else 0 
                     for i in range(len(u))], dtype=myint)

In [38]:
def agree_(u,v):
    '''
    Given two vectors u and v, return 1 iff u and v agree at all indices
    and 0 otherwise.
    '''
    ag = agree(u,v)
    return int(ag.all())

In [157]:
def agree_v(u,v):
    return not (u*v == -1).all()

In [39]:
def agree_mat(A,B):
    '''
    Given two matrices A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = np.equal(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
    result = np.prod(ag, axis=-1, dtype=myint)
    return result

In [215]:
def agree_m(A, B, axis=0):
    return (~np.equal(A*B, -1)).prod(axis=axis)

In [73]:
# twice as slow as pytorch-cpu. not worth it.
# def agree_mat_ne(A,B):
#     '''
#     Given two matrices A,B :: (n,m)
#     return C::(n,1) where 
#     C[i] = 1 iff A[i] + B[i] agree at all indices,
#     returning 0 otherwise.
#     '''

#     # (x == 0 | y == 0) | ((x != 0 & y != 0) & (x == y))
#     A_unspecified = ne.evaluate('A == 0')
#     B_unspecified = ne.evaluate('B == 0')
#     A_v_B_unspecified = ne.evaluate('A_unspecified | B_unspecified')
    
#     A_specified = ne.evaluate('A != 0')
#     B_specified = ne.evaluate('B != 0')
#     both_A_B_specified = ne.evaluate('A_specified & B_specified')
#     A_equal_B = np.equal(A,B)
#     A_B_both_specified_also_equal = ne.evaluate('both_A_B_specified & A_equal_B')
    
#     ag = ne.evaluate('A_v_B_unspecified | A_B_both_specified_also_equal')
# #     return ag
# #     last_axis = ag.ndim-1
# #     a = last_axis
# #     result = ne.evaluate('prod(ag, axis=1)').astype(myint) #this manually setting it to a specific value still causes a bizarre error, just a different one
# #     result = ne.evaluate('prod(ag, axis=a)').astype(myint) #axis can't be a variable or else it causes a bizarre error
# #     result = ne.evaluate('prod(ag, axis=last_axis)').astype(myint) #axis can't be a variable or else it causes a bizarre error
#     result = np.prod(ag, axis=-1, dtype=myint)
#     return result

In [41]:
def agree_mat_t(A,B):
    '''
    Given two matrices (torch tensors) A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = torch.eq(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
#     result = np.prod(ag, axis=-1, dtype=myint)
    result = torch.zeros([A.shape[0]], dtype=my_dtype, device=A.device)
    result = torch.prod(ag, dim=1,dtype=my_dtype, out=result)
#     result = ag.type(torch.cuda.ByteTensor).all()
    if result.device.type == 'cuda':
        torch.cuda.empty_cache()
    return result#.type(my_torch_type)

In [239]:
def agree_mt(A, B, dim=0):
    return (~torch.eq(A*B, -1 * torch.ones(A.shape, dtype=A.dtype, device=A.device))).prod(dim=dim)

In [42]:
#note that this scales *poorly* with the number of features m

# Given that each feature's value is sampled iid and uniformly,
# the probability that two randomly generated features *disagree*
# is 2/9 = p('+-' ∨ '-+'), so the probability of *agreement* is 7/9.
# Therefore the probability of two random feature vectors with m features
# agreeing on all features is (7/9)^m

(7/9)**m

def make_agreeing_vector_pair(pred=None):
    u = make_random_pfv()
    v = make_random_pfv()
    if pred is None:
        while not agree_(u,v):
            u = make_random_pfv()
            v = make_random_pfv()
        return u,v
    while not agree_(u,v) and not pred(u,v):
        u = make_random_pfv()
        v = make_random_pfv()
    return u,v

0.0030879993711559394

In [43]:
num_test_pairs = int(1e5)
# random_vector_pairs = [(make_random_pfv(), make_random_pfv()) for each in range(num_test_pairs)]
random_vector_pairs = [(choice(objects), choice(objects)) for each in range(num_test_pairs)]
len(random_vector_pairs)

100000

In [44]:
num_test_pairs = int(1e5)
# agreeing_vector_pairs = [make_agreeing_vector_pair() for each in range(num_test_pairs)]
agreeing_vector_pairs = []
for each in range(num_test_pairs):
    obj = choice(objects)
    ag_obj = gen_agreeing(obj)
    agreeing_vector_pairs.append((obj, ag_obj))
len(agreeing_vector_pairs)

100000

In [45]:
# first = lambda seq: seq[0]
# second = lambda seq: seq[1]

stack_a, stack_b = lmap(first, random_vector_pairs), lmap(second, random_vector_pairs)
random_pair_stack_a, random_pair_stack_b = np.array(stack_a), np.array(stack_b)
random_pair_stack_a.dtype
random_pair_stack_b.dtype

random_pair_stack_a_t, random_pair_stack_b_t = torch.from_numpy(random_pair_stack_a.astype(np.int32)).type(torch.int8), torch.from_numpy(random_pair_stack_b.astype(np.int32)).type(torch.int8)

if torch.cuda.is_available():
    random_pair_stack_a_tc, random_pair_stack_b_tc = random_pair_stack_a_t.cuda(), random_pair_stack_b_t.cuda()

dtype('int8')

dtype('int8')

In [46]:
stack_a, stack_b = lmap(first, agreeing_vector_pairs), lmap(second, agreeing_vector_pairs)
agreeing_pair_stack_a, agreeing_pair_stack_b = np.array(stack_a), np.array(stack_b)
agreeing_pair_stack_a.dtype
agreeing_pair_stack_b.dtype

dtype('int8')

dtype('int8')

In [60]:
%%timeit

list(starmap(agree_, random_vector_pairs));

8.24 s ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [161]:
%%timeit

list(starmap(agree_v, random_vector_pairs));

338 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [178]:
%%timeit

list(starmap(agree_m, random_vector_pairs));

466 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [183]:
assert list(starmap(agree_v, random_vector_pairs)) == list(starmap(agree_m, random_vector_pairs))

In [59]:
%%timeit

agree_mat(random_pair_stack_a, random_pair_stack_b)

5.87 ms ± 6.44 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [217]:
%%timeit

agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)

5 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [243]:
%%timeit

agree_mt(random_pair_stack_a_t, random_pair_stack_b_t, dim=1)

13.8 ms ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [244]:
if torch.cuda.is_available():
    %timeit agree_mt(random_pair_stack_a_t.cuda(), random_pair_stack_b_t.cuda(), dim=1)

826 µs ± 778 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [246]:
if torch.cuda.is_available():
    %timeit agree_mt(random_pair_stack_a_tc, random_pair_stack_b_tc, dim=1)

373 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [242]:
assert (agree_mat(random_pair_stack_a, random_pair_stack_b) == agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)).all()
assert (agree_mt(random_pair_stack_a_t, random_pair_stack_b_t, dim=1).numpy() == agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)).all()

In [58]:
# %%timeit

# agree_mat_ne(random_pair_stack_a, random_pair_stack_b)

10.9 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [247]:
%%timeit

agree_mat_t(random_pair_stack_a_t, random_pair_stack_b_t)

13.4 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [248]:
if torch.cuda.is_available():
    %timeit agree_mat_t(random_pair_stack_a_tc, random_pair_stack_b_tc)

445 µs ± 64.8 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [249]:
np.array_equal(agree_mat(random_pair_stack_a, random_pair_stack_b), 
               list(starmap(agree_, random_vector_pairs)))

True

In [250]:
n = num_test_pairs
for i in range(n):
    u = random_pair_stack_a[i]
    v = random_pair_stack_b[i]
    assert agree_(u,v) == agree_mat(u,v), '{0}, {1} -> {2} vs. {3}'.format(u,v, agree_(u,v), agree_mat(u,v, True))

In [251]:
if torch.cuda.is_available():
    agreement = agree_mat_t
else:
    agreement = agree_mat

## Union

In [79]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs

def cup(x,y):
    '''
    Formula:
    x or y, where 1 = T, -1 = T, 0 = F
    
    Algebra:
    0 is the identity ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    (-1 and +1 are mutual inverses, but this case shouldn't occur when agree(x,y) holds)
    
    Pattern:
    x ∪ x = x
    
    0 ∪ y = y
    x ∪ 0 = x
    
    _ ∪ _ = 0  \\ <- shouldn't occur in two pfvs that agree
    '''
    if x == 0:  #if x is unspecified, return y
        return y
    elif y == 0: #if y is unspecified, return x
        return x
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

for x,y in XYs:
    ((x,y), cup(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1)

((-1, 0), -1)

((-1, 1), 0)

((0, -1), -1)

((0, 0), 0)

((0, 1), 1)

((1, -1), 0)

((1, 0), 1)

((1, 1), 1)

In [80]:
def union(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.sign(u + v)

## Intersection

In [81]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs 
    
def cap(x,y):
    '''
    Algebra:
    0 is the annihilating element ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    -1 and +1 annihilate each other
    
    Pattern:
    x ∩ x = x
    
    0 ∩ _ = 0
    _ ∩ 0 = 0
    
    _ ∩ _ = 0
    '''
    if x == 0: #if x is unspecified, return 0
        return 0
    elif y == 0: #if y is unspecified, return 0
        return 0
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

def foo(x,y):
    return np.sign( (x == y) * (x + y) )

# def bar(x,y):
#     return (x == y) * (x + y) * 0.5

# def baz(x,y):
#     return (x == y) * int((x + y) / 2)

for x,y in XYs:
#     ((x,y), cap(x,y))
#     ((x,y), cap(x,y), foo(x,y), bar(x,y), baz(x,y))
    ((x,y), cap(x,y), foo(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1, -1)

((-1, 0), 0, 0)

((-1, 1), 0, 0)

((0, -1), 0, 0)

((0, 0), 0, 0)

((0, 1), 0, 0)

((1, -1), 0, 0)

((1, 0), 0, 0)

((1, 1), 1, 1)

In [82]:
def intersection(u, v):
    return np.sign(  np.equal(u, v) * (u + v) )

## Extension

In [83]:
def getIndex(o, O):
    matches = [i for i,v in enumerate(O) if np.array_equal(v,o)]
    if len(matches) == 0:
        return -1
    if CAREFUL:
        assert len(matches) == 1
    return matches[0]

In [84]:
def makeExtensionVector(positive_Indices, O):
    return np.array([1 if i in positive_Indices else 0 for i in np.arange(O.shape[0])], dtype=myint)

In [85]:
def extension(v, O, asIndexVector=True):
    '''
    The extension of a partial feature vector v is the set of object vectors
    (= fully specified, or 'total' feature vectors) that 'agree' with it.
    '''
    matches = tuple([o for o in O if agree_(v,o)])
#     matches = tuple([o for o in objects if agree(v,o).all()])
#     matches = np.array([1.0 if np.linalg.norm(agree(v,o), 1) == num_features else 0.0 for o in objects])
    if asIndexVector:
        return makeExtensionVector([getIndex(o, O) for o in matches], O)
    return matches

In [138]:
def ramp(M):
    return np.heaviside(M-1, 1).astype(myint)

def primed(p):
    mag_p = np.sum(np.abs(p))
    return p / mag_p

def extension_alt3(s, O):
    if np.array_equal(s, np.zeros(s.shape)):
        return np.ones((l,), dtype=myint)
    p = s
#     mag_p = np.sum(np.abs(p))
#     p_prime = p / mag_p
    return ramp( np.dot(O, primed(p)) )

def heaviside_t(M):
    return M >= 0

def ramp_t(M):
    return heaviside_t(M-1).type(torch.int8)

def primed_t(p):
    if p.device.type == 'cuda':
        mag_p = torch.sum(torch.abs(p.type(torch.int32)))
    else:
        mag_p = torch.sum(torch.abs(p))
    return p / mag_p

def extension_alt3_t(s, O):
    if torch.equal(s, torch.zeros(s.shape, dtype=torch.int8, device=s.device)):
        return torch.ones((l,), dtype=torch.int8)
    p = s
    #FIXME broadcasting is different in pytorch compared to numpy
    return ramp_t( torch.dot(O, primed_t(p)) )

In [87]:
def extension_(pfv, O):
    return agree_mat(pfv, O)

In [102]:
O_t = torch.from_numpy(O.astype(np.int32)).type(torch.int8)

In [103]:
def extension_t(pfv_t, O_t):
    return agree_mat_t(pfv_t, O_t)

In [309]:
def extensions_t(pfvs_t, O_t):
    return agree_mt(pfvs_t.unsqueeze(1), O_t[None, :, :], dim=2).type(torch.int8)

In [303]:
extensions_t(random_pair_stack_a_tc, O_tc)

torch.Size([100000, 91])

In [305]:
extensions_t(random_pair_stack_a_t, O_t)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cpu')

In [91]:
num_test_pairs = int(1e5)
random_vectors = [make_random_pfv() for each in tqdm(range(num_test_pairs))]
# random_vectors = [choice(objects) for each in tqdm(range(num_test_pairs))]
random_vectors_t = [torch.from_numpy(v.astype(np.int32)).type(torch.int8) for v in random_vectors]
len(random_vectors)

100%|██████████| 100000/100000 [00:00<00:00, 211766.26it/s]


100000

In [96]:
random_vectors_w_nonempty_ext = [v 
                                 for v in tqdm(random_vectors) 
                                 if extension_(v, O).sum() != 0]
len(random_vectors_w_nonempty_ext)

100%|██████████| 100000/100000 [00:02<00:00, 36319.86it/s]


4852

In [98]:
rv = choice(random_vectors_w_nonempty_ext); rv
extension(rv, O)

array([ 0,  0, -1, -1,  0,  0,  0, -1, -1,  1,  1,  1,  1, -1, -1,  0,  0,
        0,  0,  0, -1,  0,  1], dtype=int8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int8)

In [130]:
# %%timeit

#slow AF - 12m on brh
[extension(v, O) for v in tqdm(random_vectors)]






  0%|          | 0/100000 [00:00<?, ?it/s]




  0%|          | 13/100000 [00:00<13:29, 123.45it/s]




  0%|          | 27/100000 [00:00<13:13, 125.92it/s]




  0%|          | 40/100000 [00:00<13:09, 126.66it/s]




  0%|          | 54/100000 [00:00<13:00, 128.07it/s]




  0%|          | 68/100000 [00:00<12:53, 129.16it/s]




  0%|          | 82/100000 [00:00<12:41, 131.19it/s]




  0%|          | 96/100000 [00:00<12:42, 131.03it/s]




  0%|          | 110/100000 [00:00<12:35, 132.29it/s]




  0%|          | 124/100000 [00:00<12:36, 132.01it/s]




  0%|          | 138/100000 [00:01<12:33, 132.49it/s]




  0%|          | 151/100000 [00:01<12:40, 131.25it/s]




  0%|          | 165/100000 [00:01<12:38, 131.70it/s]




  0%|          | 179/100000 [00:01<12:30, 132.93it/s]




  0%|          | 193/100000 [00:01<12:27, 133.53it/s]




  0%|          | 207/100000 [00:01<12:21, 134.50it/s]




  0%|          | 221/100000 [00:01<12:23, 134.18it/s]




  0%|          | 235/100000

  2%|▏         | 1551/100000 [00:11<12:14, 134.05it/s]




  2%|▏         | 1565/100000 [00:11<12:17, 133.49it/s]




  2%|▏         | 1579/100000 [00:11<12:20, 132.82it/s]




  2%|▏         | 1593/100000 [00:11<12:21, 132.79it/s]




  2%|▏         | 1607/100000 [00:12<12:16, 133.58it/s]




  2%|▏         | 1621/100000 [00:12<12:17, 133.35it/s]




  2%|▏         | 1635/100000 [00:12<12:14, 133.99it/s]




  2%|▏         | 1649/100000 [00:12<12:12, 134.34it/s]




  2%|▏         | 1663/100000 [00:12<12:14, 133.83it/s]




  2%|▏         | 1677/100000 [00:12<12:14, 133.89it/s]




  2%|▏         | 1691/100000 [00:12<12:15, 133.60it/s]




  2%|▏         | 1705/100000 [00:12<12:20, 132.70it/s]




  2%|▏         | 1719/100000 [00:12<12:14, 133.74it/s]




  2%|▏         | 1733/100000 [00:13<12:12, 134.22it/s]




  2%|▏         | 1747/100000 [00:13<12:17, 133.30it/s]




  2%|▏         | 1761/100000 [00:13<12:18, 133.00it/s]




  2%|▏         | 1775/100000 [00:13<12:17, 133.20it/s]



KeyboardInterrupt: 

In [99]:
%%timeit

lmap(lambda v: extension_alt3(v, O), random_vectors)

2.07 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%timeit

lmap(lambda v: extension_(v,O), random_vectors)

2.32 s ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%%timeit

lmap(lambda v: extension_t(v,O_t), random_vectors_t)

5.01 s ± 75.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
if torch.cuda.is_available():
    O_tc = O_t.cuda()

In [258]:
rv_t = choice(random_vectors_t)
rv_t

tensor([ 1,  1,  0,  1, -1,  1,  1, -1, -1,  1,  1,  0,  0,  1,  1,  1, -1,  1,
        -1,  0,  1,  1, -1], device='cpu', dtype=torch.int8)

In [298]:
agree_mat_t(rv_t.cuda(), O_tc)
agree_mt(rv_t.cuda(), O_tc, dim=1)
agree_mt(rv_t.cuda(), O_tc, dim=1).shape

# agree_mt(random_pair_stack_a_tc, O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

torch.Size([91])

In [267]:
rv_t.shape
O_tc.shape
' '
random_pair_stack_a_tc.shape
O_tc.shape
# random_pair_stack_a_tc

torch.Size([23])

torch.Size([91, 23])

' '

torch.Size([100000, 23])

torch.Size([91, 23])

In [268]:
agree_mt(rv_t.cuda(), O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [269]:
agree_mt(rv_t.cuda()[None, :], O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [291]:
random_pair_stack_a_tc.shape
O_tc.shape

(random_pair_stack_a_tc.unsqueeze(1) * O_tc[None, :, :]).shape

torch.Size([100000, 23])

torch.Size([91, 23])

torch.Size([100000, 91, 23])

In [300]:
agree_mt(random_pair_stack_a_tc.unsqueeze(1), O_tc[None, :, :], dim=2).shape

torch.Size([100000, 91])

In [ ]:
# def agree_mt(A, B, dim=0):
#     return (~torch.eq(A*B, -1 * torch.ones(A.shape, dtype=A.dtype, device=A.device))).prod(dim=dim)

In [261]:
agree_mt(random_pair_stack_a_tc, random_pair_stack_b_tc, dim=1)

tensor([0, 0, 0,  ..., 0, 0, 0])

In [122]:
if torch.cuda.is_available():
    extension_t(rv_t.cuda(), O_tc)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

In [137]:
# [extension_alt3_t(v.cuda(), O_tc) for v in tqdm(random_vectors_t)]

In [124]:
# %%timeit

if torch.cuda.is_available():
    #takes 2m!!
    [extension_t(v.cuda(), O_tc) for v in tqdm(random_vectors_t)]
    # list(map(lambda v: extension_t(v.cuda(), O_tc), random_vectors_t))
    # lmap(lambda v: extension_t(v.cuda(), O_t.cuda()), random_vectors_t)


 97%|█████████▋| 96875/100000 [01:35<00:02, 1222.92it/s]


 99%|█████████▉| 98967/100000 [01:39<00:01, 616.45it/s]

KeyboardInterrupt: 

In [140]:
if torch.cuda.is_available():
    random_vectors_tc = torch.stack(random_vectors_t).cuda()
    
    len(random_vectors_t)
    random_vectors_t[0].shape
    
    random_vectors_tc.shape

In [144]:
if torch.cuda.is_available():
    
    [extension_t(v, O_tc) for v in tqdm(random_vectors_tc)]










  0%|          | 0/100000 [00:00<?, ?it/s]








  0%|          | 1/100000 [00:03<87:59:33,  3.17s/it]








  1%|          | 735/100000 [00:03<61:08:37,  2.22s/it]








  2%|▏         | 1543/100000 [00:03<42:27:11,  1.55s/it]








  2%|▏         | 2354/100000 [00:03<29:28:24,  1.09s/it]








  3%|▎         | 3166/100000 [00:03<20:27:39,  1.31it/s]








  4%|▍         | 3978/100000 [00:03<14:12:12,  1.88it/s]








  5%|▍         | 4786/100000 [00:03<9:51:35,  2.68it/s] 








  6%|▌         | 5597/100000 [00:03<6:50:38,  3.83it/s]








  6%|▋         | 6404/100000 [00:03<4:45:03,  5.47it/s]








  7%|▋         | 7217/100000 [00:04<3:17:51,  7.82it/s]








  8%|▊         | 8032/100000 [00:04<2:17:20, 11.16it/s]








  9%|▉         | 8820/100000 [00:07<1:37:24, 15.60it/s]








 10%|▉         | 9628/100000 [00:07<1:07:38, 22.27it/s]








 10%|█         | 10437/100000 [00:07<46:58, 31.78it/s] 








 11%|█         | 11244/100000 [00:08<32:3

 70%|███████   | 70056/100000 [00:48<00:59, 506.12it/s] 








 71%|███████   | 70855/100000 [00:48<00:41, 703.91it/s]








 72%|███████▏  | 71659/100000 [00:48<00:29, 969.17it/s]








 72%|███████▏  | 72463/100000 [00:48<00:20, 1316.51it/s]








 73%|███████▎  | 73270/100000 [00:48<00:15, 1757.74it/s]








 74%|███████▍  | 74062/100000 [00:48<00:11, 2292.92it/s]








 75%|███████▍  | 74863/100000 [00:49<00:08, 2917.55it/s]








 76%|███████▌  | 75658/100000 [00:49<00:06, 3600.98it/s]








 76%|███████▋  | 76459/100000 [00:49<00:05, 4312.44it/s]








 77%|███████▋  | 77252/100000 [00:49<00:04, 4995.55it/s]








 78%|███████▊  | 78038/100000 [00:52<00:33, 661.45it/s] 








 79%|███████▉  | 78794/100000 [00:53<00:23, 910.75it/s]








 80%|███████▉  | 79607/100000 [00:53<00:16, 1241.40it/s]








 80%|████████  | 80415/100000 [00:53<00:11, 1663.86it/s]








 81%|████████  | 81228/100000 [00:53<00:08, 2185.12it/s]








 82%|████████▏ | 82032/10000

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=

In [145]:
result_l = [extension_t(v, O_tc) for v in tqdm(random_vectors_tc)]










  0%|          | 0/100000 [00:00<?, ?it/s]








  0%|          | 1/100000 [00:01<47:09:17,  1.70s/it]








  1%|          | 797/100000 [00:01<32:44:48,  1.19s/it]








  2%|▏         | 1616/100000 [00:01<22:44:04,  1.20it/s]








  2%|▏         | 2431/100000 [00:01<15:46:59,  1.72it/s]








  3%|▎         | 3247/100000 [00:02<10:57:24,  2.45it/s]








  4%|▍         | 4068/100000 [00:02<7:36:20,  3.50it/s] 








  5%|▍         | 4885/100000 [00:02<5:16:46,  5.00it/s]








  6%|▌         | 5685/100000 [00:02<3:39:56,  7.15it/s]








  6%|▋         | 6450/100000 [00:06<2:35:02, 10.06it/s]








  7%|▋         | 7209/100000 [00:06<1:47:42, 14.36it/s]








  8%|▊         | 8020/100000 [00:06<1:14:47, 20.50it/s]








  9%|▉         | 8829/100000 [00:06<51:57, 29.25it/s]  








 10%|▉         | 9636/100000 [00:06<36:06, 41.72it/s]








 10%|█         | 10436/100000 [00:06<25:06, 59.46it/s]








 11%|█         | 11236/100000 [00:06<17:28, 8

 71%|███████   | 70600/100000 [00:42<00:17, 1700.09it/s]








 71%|███████▏  | 71412/100000 [00:42<00:12, 2228.70it/s]








 72%|███████▏  | 72223/100000 [00:42<00:09, 2848.31it/s]








 73%|███████▎  | 73035/100000 [00:43<00:07, 3536.91it/s]








 74%|███████▍  | 73840/100000 [00:43<00:06, 4251.83it/s]








 75%|███████▍  | 74642/100000 [00:43<00:05, 4948.95it/s]








 75%|███████▌  | 75445/100000 [00:43<00:04, 5591.36it/s]








 76%|███████▌  | 76241/100000 [00:46<00:35, 674.23it/s] 








 77%|███████▋  | 77047/100000 [00:47<00:24, 929.83it/s]








 78%|███████▊  | 77857/100000 [00:47<00:17, 1266.03it/s]








 79%|███████▊  | 78677/100000 [00:47<00:12, 1696.25it/s]








 80%|███████▉  | 79500/100000 [00:47<00:09, 2226.46it/s]








 80%|████████  | 80317/100000 [00:47<00:06, 2847.98it/s]








 81%|████████  | 81135/100000 [00:47<00:05, 3539.88it/s]








 82%|████████▏ | 81953/100000 [00:47<00:04, 4265.31it/s]








 83%|████████▎ | 82770/100

In [304]:
%%timeit

extensions_t(random_pair_stack_a_tc, O_tc)

30.9 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [307]:
extensions_t(random_pair_stack_b_tc, O_tc)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [306]:
extensions_t(random_pair_stack_b_tc, O_tc)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [312]:
extensions_t(random_pair_stack_b_t, O_t)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cpu', dtype=torch.int8)

In [310]:
extensions_t(random_pair_stack_b_tc, O_tc).shape
extensions_t(random_pair_stack_b_tc, O_tc).dtype

torch.Size([100000, 91])

torch.int8

In [ ]:
#check for equality?

In [311]:
interpretation = extension_t
interpretations = extensions_t

## Entailed pfvs

In [47]:
def specifiable_zero_indices(p, ext_p):
    '''
    Given p and A::(n,m) = ⟦p⟧:
    
    If p_j = 0 and ∀i A_{i,j} = k≠0, then
    p_j is unspecified (i.e. p_j = 0) but 
    can be set to k and yield a co-extensive 
    and more specific pfv p'. (NB: p' entails 
    p.)
    
    This function returns a list of (index, value) pairs
    indicating the set of 0-valued indices of p that can 
    be specified, plus what the common value at that index is.
    
    Correctly specifying any one or any combination
    of the indices in this list of indices will result
    in a more specific vector than p that is coextensive.
    
    From this list, you can construct (or count) all of the
    more specified pfvs that are coextensive with p.
    '''
    A = ext_p
    n = A.shape[0]
    if n == 0:
        return set()
    n_opp = -1.0 * n
#     zeros = np.nonzero(p)[0]
    zero_indices = np.array(tuple(  set(range(len(p))) - set(np.nonzero(p)[0])  ), dtype=myint)
    specifiable_indices = set()
    for j in zero_indices:
        j_col_sum = np.sum(A[:,j])
        if j_col_sum == n:
            specifiable_indices.add((j, 1))
        if j_col_sum == n_opp:
            specifiable_indices.add((j, -1))
    return specifiable_indices

def specify(p, specs):
    '''
    Given a partial feature vector p and a set of
        (index i, non-zero value v)
    pairs where p_i ≠ 0, returns a more specific p'
    where p'_i = v as indicated by spec.
    '''
    p_prime = p.copy()
    for i,v in specs:
        p_prime[i] = v
    return p_prime

def entailed_pfvs(p, O, no_total_fvs = True):
    '''
    Given a partial feature vector p and a set of objects
    (total feature vectors) O, this returns the set of
    partial feature vectors that are strictly more specific
    than p that have the same extension in O.
    '''
    x_p = np.array(extension(p, O, False))
    specifiable_indices = specifiable_zero_indices(p, x_p)
    num_specifiable_indices = len(specifiable_indices)
    specifications = {tuple(combinations(specifiable_indices, r) )
                      for r in range(1, num_specifiable_indices+1)}
    entailed_vectors = np.array([specify(p, spec)
                                 for r_level in specifications 
                                 for spec in r_level], dtype=myint)
    if not no_total_fvs:
        return entailed_vectors
    entailed_pfvs = np.array([v for v in entailed_vectors
                              if len(v.nonzero()[0]) < m])
    return entailed_pfvs

# Generation of $S_i$: all pfvs with exactly $i$ specified values

In [52]:
# from functools import reduce

In [48]:
def grand_union(pfvs):
    return reduce(union, pfvs)

In [49]:
def one_hot_stack(indices):
#     n_values = np.max(indices) + 1
#     n_values = num_features
    n_values = m
    return np.eye(n_values,dtype=myint)[indices] 

In [50]:
def indexChoicesToComponentOptions(index_choices):
    indices = list(index_choices)
    one_hots = one_hot_stack(indices)
#     component_options = tuple([(v, -1 * v) for v in one_hots])
    component_options = ((v, -1 * v) for v in one_hots)
    return component_options

def componentOptionsToChoices(component_options):
#     choice_combinations = tuple(product(*component_options))
    choice_combinations = product(*component_options)
#     return tuple(starmap(union,
#                          choice_combinations))
#     return tuple(map(grand_union,
#                      choice_combinations))
    return map(grand_union, choice_combinations)

def make_Si_naive(i):
    index_choices = combinations(range(m), i)
    componentOptions = (indexChoicesToComponentOptions(c) for c in index_choices)
    componentChoices = (componentOptionsToChoices(o) for o in componentOptions)
#     choices_flattened = reduce(lambda a,b: a + b, componentChoices)
    choices_flattened = tuple(reduce(lambda a,b: chain.from_iterable([a,b]), componentChoices))
    return np.array(choices_flattened)

In [51]:
construct_Si = make_Si_naive

In [52]:
# calculate_Xi = interpretation

#FIXME this can/should be parallelized and memory mapped
def calculate_Xi_naive(Si, O):
    return np.array([interpretation(p, O) for p in Si], dtype=myint)

In [53]:
def heaviside(x):
    return np.array(1 * (x >= 0))

def extension_multi_bool(p_mat,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (M,num_p) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    K_vec = np.sum(abs(p_mat),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat) #shape is (L,num_p)
    return heaviside(E-K_vec[np.newaxis,:])

# calculate_Xi = extension_multi_bool

def calculate_Xi(p_mat, V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (num_p, M) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    p_mat_prime = p_mat.T
    K_vec = np.sum(abs(p_mat_prime),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat_prime) #shape is (L,num_p)
    result = heaviside(E-K_vec[np.newaxis,:]).T
    
#     K_vec_prime = np.sum(abs(p_mat), axis=1)
# #     assert np.array_equal(K_vec_prime, K_vec.T)
#     E_prime = np.dot(p_mat, V.T)
# #     assert np.array_equal(E_prime, E.T)
#     result_prime = heaviside(E_prime-K_vec_prime[:,np.newaxis])
#     assert result_prime.shape == result.shape, '{0} vs. {1}'.format(result_prime.shape, result.shape)
#     assert np.array_equal(result_prime, result.T)    
    return result#_prime


In [54]:
O.shape

(12, 5)

In [55]:
m

5

In [56]:
S3 = construct_Si(3)
S3.shape

(80, 5)

In [57]:
S3.T.shape

(5, 80)

In [58]:
O.shape

(12, 5)

In [59]:
%%timeit

calculate_Xi_naive(S3, O)

1.11 ms ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [148]:
%%timeit

calculate_Xi(S3, O)

18.2 µs ± 58.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [60]:
calculate_Xi_naive(S3, O).shape
calculate_Xi_naive(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [61]:
calculate_Xi(S3, O).shape
calculate_Xi(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

# Generate $\overline{S}_i$, $\overline{X}_i$ by removing vectors with empty extension in $S_i$ + their empty extension in $X_i$

In [58]:
EMPTY = np.zeros((l,), dtype=myint)

In [73]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_naive(Si, Xi):
    return np.array([v for i,v in enumerate(Si) 
#                      if not empty_extension(Xj[i])])
                     if not np.array_equal(EMPTY, Xi[i])])

In [71]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_Xi_bar_naive(Si, Xi):
    non_empty_indices = np.array([i for i,v in enumerate(Si)
                                  if not np.array_equal(EMPTY, Xi[i])])
    Si_bar = np.array([Si[i] for i in non_empty_indices])
    Xi_bar = np.array([Xi[i] for i in non_empty_indices])
    return Si_bar, Xi_bar

In [75]:
def make_Si_bar_Xi_bar_alt(Si, Xi):
#     non_empty_extension_row_indices = np.array([i for i,v in enumerate(Si)
#                                                 if np.sum(Xi[i]) != 0])
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    
    Si_bar = Si[non_empty_extension_row_indices,:]
    Xi_bar = Xi[non_empty_extension_row_indices,:]
    return Si_bar, Xi_bar

In [76]:
construct_Si_bar = make_Si_bar_naive
construct_Si_Xi_bar = make_Si_bar_Xi_bar_alt

In [172]:
S3.shape

(80, 5)

In [110]:
X3 = calculate_Xi_naive(S3, O)

In [159]:
X3.shape

(80, 9)

In [163]:
l, m

(9, 5)

In [162]:
O.shape

(9, 5)

In [169]:
np.sum(X3, axis=1).shape

(80,)

In [175]:
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).shape
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).nonzero()[0]

(80,)

array([ 0,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 18, 19, 20,
       21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40,
       42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61,
       62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 75, 77, 78])

In [168]:
non_empty_row_indices = np.array([i for i,v in enumerate(S3)
                                  if np.sum(X3[i]) != 0])
non_empty_row_indices.shape

(64,)

In [157]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[0], make_Si_bar_Xi_bar_alt(S3, X3)[0] )

True

In [158]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[1], make_Si_bar_Xi_bar_alt(S3, X3)[1] )

True

In [177]:
%%timeit

make_Si_bar_Xi_bar_naive(S3, X3)

286 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [178]:
%%timeit

make_Si_bar_Xi_bar_alt(S3, X3)

9.55 µs ± 55.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Convert $\overline{X}_i$ to a sparse representation

In [62]:
# import sparse

In [78]:
def density(a):
    num_cells = reduce(lambda x,y: x * y, a.shape)
    d = len(np.nonzero(a)[0]) / num_cells
    return d

def sparsity(a):
    return 1 - density(a)

In [79]:
def to_sparse(v):
    return sparse.COO(v)

# Local processing pipeline to generate $\overline{S}_i$ and (dense) $\overline{X}_i$, $\forall i$ 

In [68]:
def construct_Si_bar_Xi_bar(i, O):
    Si = construct_Si(i)
    Xi = calculate_Xi(Si, O)
    Si_bar, Xi_bar = construct_Si_Xi_bar(Si, Xi)
#     Si_bar = construct_Si_bar(Si, Xi)
#     del Si
#     del Xi
    #FIXME you shouldn't have to recalculate the extensions of everything in Si_bar!
#     Xi_bar = calculate_Xi(Si_bar, O)
    return Si_bar, Xi_bar #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [63]:
construct_Si(1)

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [80]:
S1_bar, X1_bar = construct_Si_bar_Xi_bar(1, O)
sparsity(S1_bar)
sparsity(X1_bar)

0.8

0.5

In [81]:
S1_bar

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [82]:
S2_bar, X2_bar = construct_Si_bar_Xi_bar(2, O)
sparsity(S2_bar)
sparsity(X2_bar)

0.6

0.75

In [83]:
S3_bar, X3_bar = construct_Si_bar_Xi_bar(3, O)
sparsity(S3_bar)
sparsity(X3_bar)

0.4

0.8529411764705882

In [84]:
S4_bar, X4_bar = construct_Si_bar_Xi_bar(4, O)
sparsity(S4_bar)
sparsity(X4_bar)

0.19999999999999996

0.8979591836734694

In [85]:
S5_bar, X5_bar = construct_Si_bar_Xi_bar(5, O)
sparsity(S5_bar)
sparsity(X5_bar)

0.0

0.9166666666666666

# Memory-mapping

In [113]:
X5_bar_sparse = to_sparse(X5_bar)

In [114]:
X5_bar_sparse.nbytes

288

In [155]:
def construct_Si_bar_Xi_bar_mmap(i, O):
    s = ''
#     print('i = {0}'.format(i))
    s += 'i = {0}'.format(i) + '\n'
    
    Si_fn = 'S{0}.dat'.format(i)
    n_pfvs = int(binom(m, i) * (2 ** i))
    n_features = m
    Si_shape = (n_pfvs, n_features)
    
#     print('Si_shape: {0}'.format(Si_shape))
    s += 'Si_shape: {0}'.format(Si_shape) + '\n'
    
    Si = np.memmap(Si_fn, dtype=myint, mode='w+', shape = Si_shape)
#     Si = construct_Si(i)
    Si[:] = construct_Si(i)
#     print('Finished writing S{0} to disk as {1} w/ {2} GB'.format(i, Si_fn, Si.nbytes / 1e9))
    s += 'Finished writing S{0} to disk as {1} w/ {2} GB'.format(i, Si_fn, Si.nbytes / 1e9) + '\n'
    
    Xi_fn = 'X{0}.dat'.format(i)
    l = O.shape[0]
    n_objects = l
    Xi_shape = (n_pfvs, n_objects)
    Xi = np.memmap(Xi_fn, dtype=myint, mode='w+', shape = Xi_shape)
    Xi[:] = calculate_Xi(Si, O)
#     print('Finished writing X{0} to disk as {1} w/ {2} GB'.format(i, Xi_fn, Xi.nbytes / 1e9))
#     print(' ')
    s += 'Finished writing X{0} to disk as {1} w/ {2} GB'.format(i, Xi_fn, Xi.nbytes / 1e9) + '\n\n'
    
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    num_ne_pfvs = len(non_empty_extension_row_indices)
#     print('\tFinished identifying {0} pfvs with non-empty extensions.'.format(num_ne_pfvs))
    s += '\tFinished identifying {0} pfvs with non-empty extensions.'.format(num_ne_pfvs) + '\n'
    
    Si_bar_fn = 'S{0}_bar.dat'.format(i)
    Si_bar_shape = (num_ne_pfvs, n_features)
    Si_bar = np.memmap(Si_bar_fn, dtype=myint, mode='w+', shape = Si_bar_shape)
    Si_bar[:] = Si[non_empty_extension_row_indices,:]
#     print('\tFinished writing S{0}_bar to disk as {1} w/ {2} GB'.format(i, Si_bar_fn, Si_bar.nbytes / 1e9))
#     print('\t\tSparsity of S{0}_bar: {1}'.format(i, sparsity(Si_bar)))
#     print('\t\tDeleting S{0} from disk, freeing {1} GB'.format(i, Si.nbytes / 1e9))
    s += '\tFinished writing S{0}_bar to disk as {1} w/ {2} GB'.format(i, Si_bar_fn, Si_bar.nbytes / 1e9) + '\n'
    s += '\t\tSparsity of S{0}_bar: {1}'.format(i, sparsity(Si_bar)) + '\n'
    s += '\t\tDeleting S{0} from disk, freeing {1} GB'.format(i, Si.nbytes / 1e9) + '\n'
    os.remove(Si_fn)
#     print('\t\tS{0} deleted'.format(i))
    s += '\t\tS{0} deleted'.format(i) + '\n'
#     print('{0} GB used for S{1}_bar'.format(Si_bar.nbytes / 1e9, i))
    s += '{0} GB used for S{1}_bar'.format(Si_bar.nbytes / 1e9, i) + '\n'
    
    Xi_bar_fn = 'X{0}_bar.dat'.format(i)
    Xi_bar_shape = (num_ne_pfvs, n_objects)
    Xi_bar = np.memmap(Xi_bar_fn, dtype=myint, mode='w+', shape = Xi_bar_shape)
    Xi_bar[:] = Xi[non_empty_extension_row_indices,:]
    print('\tFinished writing X{0}_bar to disk as {1} w/ {2} GB'.format(i, Xi_bar_fn, Xi_bar.nbytes / 1e9))
    s += '\tFinished writing X{0}_bar to disk as {1} w/ {2} GB'.format(i, Xi_bar_fn, Xi_bar.nbytes / 1e9) + '\n'
#     print('\t\tDeleting X{0} from disk, freeing {1} GB'.format(i, Xi.nbytes / 1e9))
    s += '\t\tDeleting X{0} from disk, freeing {1} GB'.format(i, Xi.nbytes / 1e9) + '\n'
    os.remove(Xi_fn)
#     print('\t\tX{0} deleted'.format(i))
    s += '\t\tX{0} deleted'.format(i) + '\n'
    
#     print('\t\tSparsity of X{0}_bar: {1}'.format(i, sparsity(Xi_bar)))
    s += '\t\tSparsity of X{0}_bar: {1}'.format(i, sparsity(Xi_bar)) + '\n'
#     print('\t\tCreating sparse version of X{0}_bar'.format(i))
    s += '\t\tCreating sparse version of X{0}_bar'.format(i) + '\n'
    Xi_bar_sparse_fn = 'X{0}_bar.sparse'.format(i)
    Xi_bar_sparse = to_sparse(Xi_bar)
#     print('\t\tSaving as {0}'.format(Xi_bar_sparse_fn))
    s += '\t\tSaving as {0}'.format(Xi_bar_sparse_fn) + '\n'
    sparse.save_npz(Xi_bar_sparse_fn, Xi_bar_sparse)
#     print('\t\tSaved, using {0} GB'.format(Xi_bar_sparse.nbytes / 1e9))
    s += '\t\tSaved, using {0} GB'.format(Xi_bar_sparse.nbytes / 1e9) + '\n'
#     print('\t\tDeleting {0}, saving {1} GB'.format(Xi_bar_fn, Xi_bar.nbytes / 1e9))
    s += '\t\tDeleting {0}, saving {1} GB'.format(Xi_bar_fn, Xi_bar.nbytes / 1e9) + '\n'
    
#     print('{0} GB used for sparse X{1}_bar'.format(Xi_bar_sparse.nbytes / 1e9, i))
    s += '{0} GB used for sparse X{1}_bar'.format(Xi_bar_sparse.nbytes / 1e9, i) + '\n'
    
#     print(' ')
    s += '\n'
    print(s)
    return Si_bar, Xi_bar_sparse #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [127]:
m

5

In [131]:
for i in range(1,m+1):
    print('i = {0}'.format(i))
    construct_Si_bar_Xi_bar_mmap(i, O)
    print(' ')

i = 1
Si_shape: (10, 5)
Finished writing S1 to disk as S1.dat w/ 5e-08 GB
Finished writing X1 to disk as X1.dat w/ 1.2e-07 GB
Finished identifying 10 pfvs with non-empty extensions.
Finished writing S1_bar to disk as S1_bar.dat w/ 5e-08 GB
Sparsity of S1_bar: 0.8
Deleting S1 from disk, freeing 5e-08 GB
S1 deleted
Finished writing X1_bar to disk as X1_bar.dat w/ 1.2e-07 GB
Deleting X1 from disk, freeing 1.2e-07 GB
X1 deleted
Sparsity of X1_bar: 0.5
Creating sparse version of X1_bar
Saving as X1_bar.sparse
Saved, using 1.02e-06 GB
Deleting X1_bar.dat, saving 1.2e-07 GB


(memmap([[ 1,  0,  0,  0,  0],
         [-1,  0,  0,  0,  0],
         [ 0,  1,  0,  0,  0],
         [ 0, -1,  0,  0,  0],
         [ 0,  0,  1,  0,  0],
         [ 0,  0, -1,  0,  0],
         [ 0,  0,  0,  1,  0],
         [ 0,  0,  0, -1,  0],
         [ 0,  0,  0,  0,  1],
         [ 0,  0,  0,  0, -1]], dtype=int8),
 <COO: shape=(10, 12), dtype=int8, nnz=60, fill_value=0>)

 
i = 2
Si_shape: (40, 5)
Finished writing S2 to disk as S2.dat w/ 2e-07 GB
Finished writing X2 to disk as X2.dat w/ 4.8e-07 GB
Finished identifying 40 pfvs with non-empty extensions.
Finished writing S2_bar to disk as S2_bar.dat w/ 2e-07 GB
Sparsity of S2_bar: 0.6
Deleting S2 from disk, freeing 2e-07 GB
S2 deleted
Finished writing X2_bar to disk as X2_bar.dat w/ 4.8e-07 GB
Deleting X2 from disk, freeing 4.8e-07 GB
X2 deleted
Sparsity of X2_bar: 0.75
Creating sparse version of X2_bar
Saving as X2_bar.sparse
Saved, using 2.04e-06 GB
Deleting X2_bar.dat, saving 4.8e-07 GB


(memmap([[ 1,  1,  0,  0,  0],
         [ 1, -1,  0,  0,  0],
         [-1,  1,  0,  0,  0],
         [-1, -1,  0,  0,  0],
         [ 1,  0,  1,  0,  0],
         [ 1,  0, -1,  0,  0],
         [-1,  0,  1,  0,  0],
         [-1,  0, -1,  0,  0],
         [ 1,  0,  0,  1,  0],
         [ 1,  0,  0, -1,  0],
         [-1,  0,  0,  1,  0],
         [-1,  0,  0, -1,  0],
         [ 1,  0,  0,  0,  1],
         [ 1,  0,  0,  0, -1],
         [-1,  0,  0,  0,  1],
         [-1,  0,  0,  0, -1],
         [ 0,  1,  1,  0,  0],
         [ 0,  1, -1,  0,  0],
         [ 0, -1,  1,  0,  0],
         [ 0, -1, -1,  0,  0],
         [ 0,  1,  0,  1,  0],
         [ 0,  1,  0, -1,  0],
         [ 0, -1,  0,  1,  0],
         [ 0, -1,  0, -1,  0],
         [ 0,  1,  0,  0,  1],
         [ 0,  1,  0,  0, -1],
         [ 0, -1,  0,  0,  1],
         [ 0, -1,  0,  0, -1],
         [ 0,  0,  1,  1,  0],
         [ 0,  0,  1, -1,  0],
         [ 0,  0, -1,  1,  0],
         [ 0,  0, -1, -1,  0],
        

 
i = 3
Si_shape: (80, 5)
Finished writing S3 to disk as S3.dat w/ 4e-07 GB
Finished writing X3 to disk as X3.dat w/ 9.6e-07 GB
Finished identifying 68 pfvs with non-empty extensions.
Finished writing S3_bar to disk as S3_bar.dat w/ 3.4e-07 GB
Sparsity of S3_bar: 0.4
Deleting S3 from disk, freeing 4e-07 GB
S3 deleted
Finished writing X3_bar to disk as X3_bar.dat w/ 8.16e-07 GB
Deleting X3 from disk, freeing 9.6e-07 GB
X3 deleted
Sparsity of X3_bar: 0.8529411764705882
Creating sparse version of X3_bar
Saving as X3_bar.sparse
Saved, using 2.04e-06 GB
Deleting X3_bar.dat, saving 8.16e-07 GB


(memmap([[ 1,  1,  1,  0,  0],
         [ 1,  1, -1,  0,  0],
         [ 1, -1,  1,  0,  0],
         [ 1, -1, -1,  0,  0],
         [-1,  1, -1,  0,  0],
         [-1, -1,  1,  0,  0],
         [ 1,  1,  0,  1,  0],
         [ 1,  1,  0, -1,  0],
         [ 1, -1,  0,  1,  0],
         [ 1, -1,  0, -1,  0],
         [-1,  1,  0, -1,  0],
         [-1, -1,  0,  1,  0],
         [ 1,  1,  0,  0,  1],
         [ 1,  1,  0,  0, -1],
         [ 1, -1,  0,  0,  1],
         [ 1, -1,  0,  0, -1],
         [-1,  1,  0,  0,  1],
         [-1, -1,  0,  0, -1],
         [ 1,  0,  1,  1,  0],
         [ 1,  0,  1, -1,  0],
         [ 1,  0, -1,  1,  0],
         [ 1,  0, -1, -1,  0],
         [-1,  0,  1,  1,  0],
         [-1,  0, -1, -1,  0],
         [ 1,  0,  1,  0,  1],
         [ 1,  0,  1,  0, -1],
         [ 1,  0, -1,  0,  1],
         [ 1,  0, -1,  0, -1],
         [-1,  0,  1,  0, -1],
         [-1,  0, -1,  0,  1],
         [ 1,  0,  0,  1,  1],
         [ 1,  0,  0,  1, -1],
        

 
i = 4
Si_shape: (80, 5)
Finished writing S4 to disk as S4.dat w/ 4e-07 GB
Finished writing X4 to disk as X4.dat w/ 9.6e-07 GB
Finished identifying 49 pfvs with non-empty extensions.
Finished writing S4_bar to disk as S4_bar.dat w/ 2.45e-07 GB
Sparsity of S4_bar: 0.19999999999999996
Deleting S4 from disk, freeing 4e-07 GB
S4 deleted
Finished writing X4_bar to disk as X4_bar.dat w/ 5.88e-07 GB
Deleting X4 from disk, freeing 9.6e-07 GB
X4 deleted
Sparsity of X4_bar: 0.8979591836734694
Creating sparse version of X4_bar
Saving as X4_bar.sparse
Saved, using 1.02e-06 GB
Deleting X4_bar.dat, saving 5.88e-07 GB


(memmap([[ 1,  1,  1,  1,  0],
         [ 1,  1,  1, -1,  0],
         [ 1,  1, -1,  1,  0],
         [ 1, -1,  1,  1,  0],
         [ 1, -1,  1, -1,  0],
         [ 1, -1, -1,  1,  0],
         [ 1, -1, -1, -1,  0],
         [-1,  1, -1, -1,  0],
         [-1, -1,  1,  1,  0],
         [ 1,  1,  1,  0,  1],
         [ 1,  1,  1,  0, -1],
         [ 1,  1, -1,  0,  1],
         [ 1,  1, -1,  0, -1],
         [ 1, -1,  1,  0,  1],
         [ 1, -1,  1,  0, -1],
         [ 1, -1, -1,  0,  1],
         [ 1, -1, -1,  0, -1],
         [-1,  1, -1,  0,  1],
         [-1, -1,  1,  0, -1],
         [ 1,  1,  0,  1,  1],
         [ 1,  1,  0,  1, -1],
         [ 1,  1,  0, -1,  1],
         [ 1,  1,  0, -1, -1],
         [ 1, -1,  0,  1,  1],
         [ 1, -1,  0,  1, -1],
         [ 1, -1,  0, -1,  1],
         [ 1, -1,  0, -1, -1],
         [-1,  1,  0, -1,  1],
         [-1, -1,  0,  1, -1],
         [ 1,  0,  1,  1,  1],
         [ 1,  0,  1,  1, -1],
         [ 1,  0,  1, -1,  1],
        

 
i = 5
Si_shape: (32, 5)
Finished writing S5 to disk as S5.dat w/ 1.6e-07 GB
Finished writing X5 to disk as X5.dat w/ 3.84e-07 GB
Finished identifying 12 pfvs with non-empty extensions.
Finished writing S5_bar to disk as S5_bar.dat w/ 6e-08 GB
Sparsity of S5_bar: 0.0
Deleting S5 from disk, freeing 1.6e-07 GB
S5 deleted
Finished writing X5_bar to disk as X5_bar.dat w/ 1.44e-07 GB
Deleting X5 from disk, freeing 3.84e-07 GB
X5 deleted
Sparsity of X5_bar: 0.9166666666666666
Creating sparse version of X5_bar
Saving as X5_bar.sparse
Saved, using 2.04e-07 GB
Deleting X5_bar.dat, saving 1.44e-07 GB


(memmap([[ 1,  1,  1,  1,  1],
         [ 1,  1,  1, -1,  1],
         [ 1,  1,  1, -1, -1],
         [ 1,  1, -1,  1,  1],
         [ 1,  1, -1,  1, -1],
         [ 1, -1,  1,  1, -1],
         [ 1, -1,  1, -1,  1],
         [ 1, -1, -1,  1,  1],
         [ 1, -1, -1,  1, -1],
         [ 1, -1, -1, -1, -1],
         [-1,  1, -1, -1,  1],
         [-1, -1,  1,  1, -1]], dtype=int8),
 <COO: shape=(12, 12), dtype=int8, nnz=12, fill_value=0>)

In [156]:
m = 23

max_num_objects = 2 ** m
max_num_objects
# actual_num_objects = np.random.randint(max_num_objects)
actual_num_objects = 96
actual_num_objects

assert actual_num_objects < max_num_objects

8388608

96

In [157]:
O = makeRandomObjects(actual_num_objects, m, True)
O
O.shape
l = len(O); l

array([[-1,  1, -1, ...,  1,  1, -1],
       [ 1, -1, -1, ..., -1, -1,  1],
       [-1, -1,  1, ...,  1, -1,  1],
       ..., 
       [ 1, -1,  1, ...,  1,  1,  1],
       [ 1,  1, -1, ..., -1,  1, -1],
       [-1, -1,  1, ..., -1,  1,  1]])

(96, 23)

96

In [158]:
J
V

-1

10

In [159]:
# for i in range(1,m+1):
#     print('i = {0}'.format(i))
#     construct_Si_bar_Xi_bar_mmap(i, O)
#     print(' ')

In [ ]:
par(delayed(construct_Si_bar_Xi_bar_mmap)(i, O) for i in range(1, m+1))

i = 2
i = 1
i = 3
i = 4
Si_shape: (46, 23)
Si_shape: (1012, 23)
i = 5
Si_shape: (14168, 23)
Si_shape: (141680, 23)
i = 6
i = 7
Si_shape: (1076768, 23)
Si_shape: (6460608, 23)
i = 8
i = 9
i = 10
Si_shape: (31380096, 23)
Si_shape: (418401280, 23)
Si_shape: (125520384, 23)
i = 11
i = 12
Finished writing S1 to disk as S1.dat w/ 1.058e-06 GB
i = 13
i = 14
Si_shape: (2769055744, 23)
i = 15
Si_shape: (1171523584, 23)
Si_shape: (13388840960, 23)
i = 16
Si_shape: (5538111488, 23)
Si_shape: (9372188672, 23)
i = 17
Si_shape: (16066609152, 23)
Si_shape: (16066609152, 23)
i = 18
i = 21
Si_shape: (8820883456, 23)
i = 19
i = 22
Si_shape: (530579456, 23)
Si_shape: (13231325184, 23)
Si_shape: (4642570240, 23)
Si_shape: (96468992, 23)
i = 20
i = 23
Si_shape: (1857028096, 23)
Si_shape: (8388608, 23)
Finished writing S2 to disk as S2.dat w/ 2.3276e-05 GB
Finished writing X1 to disk as X1.dat w/ 4.416e-06 GB
 
	Finished identifying 46 pfvs with non-empty extensions.
	Finished writing S1_bar to disk as S1_b

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0946s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 out of  23 | elapsed:    0.1s remaining:    1.2s


Finished writing S3 to disk as S3.dat w/ 0.000325864 GB
Finished writing X3 to disk as X3.dat w/ 0.001360128 GB
 
	Finished identifying 14168 pfvs with non-empty extensions.
	Finished writing S3_bar to disk as S3_bar.dat w/ 0.000325864 GB
		Sparsity of S3_bar: 0.8695652173913043
		Deleting S3 from disk, freeing 0.000325864 GB
		S3 deleted
0.000325864 GB used for S0.000325864_bar
	Finished writing X3_bar to disk as X3_bar.dat w/ 0.001360128 GB
		Deleting X3 from disk, freeing 0.001360128 GB
		X3 deleted
		Sparsity of X3_bar: 0.875
		Creating sparse version of X3_bar
		Saving as X3_bar.sparse
		Saved, using 0.002890272 GB
		Deleting X3_bar.dat, saving 0.001360128 GB
0.002890272 GB used for sparse X0.002890272_bar
 
Finished writing S4 to disk as S4.dat w/ 0.00325864 GB
Finished writing X4 to disk as X4.dat w/ 0.01360128 GB
 
	Finished identifying 141387 pfvs with non-empty extensions.
	Finished writing S4_bar to disk as S4_bar.dat w/ 0.003251901 GB
		Sparsity of S4_bar: 0.826086956521739

[Parallel(n_jobs=-1)]: Done   5 out of  23 | elapsed:  4.1min remaining: 14.7min


Finished writing S22 to disk as S22.dat w/ 2.218786816 GB
